<a href="https://colab.research.google.com/github/kash2023k/Pytorch-Project1/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing all necessary library for CNN
from keras.models import Sequential
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from keras.layers import Dense, Conv2D, Flatten

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
# Convert MNIST image files into Tensor of 4 dimensions (# of images , height , width , color channels)

transform = transforms.ToTensor()

In [4]:
# Train Data

train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.95MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 124kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.59MB/s]


In [5]:
# Test Data

test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
# Lets create a batch size of images .. 10

trainer_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False) # We dont want to Shuffle

In [9]:
# Define CNN Model
# Describe CNN layer - 2 layers

conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)


In [14]:
# Lets grab one MNIST Record/image

for i , (X_train, y_train) in enumerate(train_data):
    break


In [15]:
X_train.shape

torch.Size([1, 28, 28])

In [17]:
x = X_train.view(1,1,28,28) # You can use reshape also


In [18]:
# Perform out first convolution

x = F.relu(conv1(x)) #Rectified Linear Unit for our activation function

In [19]:
x.shape # 1 - Single Image , 6- filters , but the features have reduced from 28x28 to 26x26

torch.Size([1, 6, 26, 26])